In [ ]:
%matplotlib inline
from IPython.display import display, Markdown
from datetime import time, datetime, date, timedelta
import snakemd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as dates

import fitfile
from garmindb import GarminConnectConfigManager
from garmindb.garmindb import GarminDb, Attributes, ActivitiesDb, Activities, StepsActivities, ActivityLaps, ActivityRecords
from idbutils.list_and_dict import list_not_none

from jupyter_funcs import format_number

gc_config = GarminConnectConfigManager()
db_params_dict = gc_config.get_db_params()

garmin_db = GarminDb(db_params_dict)
garmin_act_db = ActivitiesDb(db_params_dict)
measurement_system = Attributes.measurements_type(garmin_db)
unit_strings = fitfile.units.unit_strings[measurement_system]
distance_units = {"kilometers": "km"}[unit_strings[fitfile.units.UnitTypes.distance_long]]

def __format_activity(activity):
    if activity:
        return [activity.activity_id, activity.name, activity.start_time.strftime("%y%m%d"), activity.sport, format_number(activity.distance, 1), activity.elapsed_time, activity.moving_time, format_number(activity.avg_speed, 1), format_number(activity.calories), activity.training_load, activity.training_effect, activity.anaerobic_training_effect]
    return ['', '', '', '', '', '', '', '', '', '', '', '']


activities = Activities.get_latest(garmin_act_db, Activities.row_count(garmin_act_db))

In [ ]:
def debug(name=None):
    doc = snakemd.new_doc()
    if (name is None):
        rows = [__format_activity(activity) for activity in activities]
    else:
        rows = [__format_activity(activity) for activity in activities if ((activity.name is not None) and (name in activity.name.lower()))]
    doc.add_heading("All Recorded Activities", 3)
    doc.add_table(['Id', 'Name', 'Date', 'Sport', f'Dist ({distance_units})', 'Elapsed Time', f'Moving Time', f'Speed ({unit_strings[fitfile.units.UnitTypes.speed]})', 'Calories', 'Exercise Load', 'Aerobic effect', 'Anaerobic effect'], rows)
    display(Markdown(str(doc)))

In [ ]:
import numpy as np

def __plot_activity(name:str, ydata:str, cdata:str, ylimdata = None, yticksdata:tuple[float, float] = None):
    start_date_array = []
    moving_time_array = []
    distance_array = []
    calorie_array = []
    time100m_array = []
    time1000m_array = []
    speed_array = []
    training_load_array = []
    training_effect_array = []
    anaerobic_training_effect_array = []
    for activity in activities:
        if ((activity.name is not None) and (name in activity.name.lower())):
            s = activity.distance
            t = (datetime.combine(date.min, activity.moving_time) - datetime.min).total_seconds()
            if (s > 0):
                time100m_array += [t/s/10]
                time1000m_array += [t/s]
            else:
                time100m_array += [0]
                time1000m_array += [0]
            start_date_array += [activity.start_time]
            moving_time_array += [t/60] # Convert seconds to minutes
            calorie_array += [activity.calories]
            distance_array += [s]
            speed_array += [activity.avg_speed]
            training_load_array += [activity.training_load]
            training_effect_array += [activity.training_effect]
            anaerobic_training_effect_array += [activity.anaerobic_training_effect]

    yarray = []
    carray = []
    match ydata:
        case "Moving Time": yarray = moving_time_array
        case "Calorie": yarray = calorie_array
        case "Distance": yarray = distance_array
        case "100m Time": yarray = time100m_array
        case "1k Time": yarray = time1000m_array
        case "Speed": yarray = speed_array
        case "Exercise Load": yarray = training_load_array
        case "Training Effect": yarray = training_effect_array
        case "Anaerobic Training Effect": yarray = anaerobic_training_effect_array
    match cdata:
        case "Moving Time": carray = moving_time_array
        case "Calorie": carray = calorie_array
        case "Distance": carray = distance_array
        case "100m Time": carray = time100m_array
        case "1k Time": carray = time1000m_array
        case "Speed": carray = speed_array
        case "Exercise Load": carray = training_load_array
        case "Training Effect": carray = training_effect_array
        case "Anaerobic Training Effect": carray = anaerobic_training_effect_array

    fig = plt.figure(figsize=(20,5), dpi= 100, facecolor='w', edgecolor='k')
    plt.scatter(start_date_array, yarray, c=carray, cmap="rainbow")

    # First remove any NaN/inf values
    x_num = dates.date2num(start_date_array)
    x_num = np.array(x_num)
    yarray = np.array(yarray)
    carray = np.array(carray)
    valid_idx = np.isfinite(x_num) & np.isfinite(yarray)
    x_clean, y_clean = x_num[valid_idx], yarray[valid_idx]

    # Filter outliers before trend calculation
    initial_trend = np.polyfit(x_clean, y_clean, 3)
    initial_fit = np.poly1d(initial_trend)
    residuals = y_clean - initial_fit(x_clean)

    # Keep 95% of points closest to the trend
    residual_threshold = np.percentile(np.abs(residuals), 95)
    mask = np.abs(residuals) <= residual_threshold

    # Fit final trend on filtered data
    trend = np.polyfit(x_num[mask], yarray[mask], 3)
    fit = np.poly1d(trend)
    x_fit = np.linspace(x_num.min(), x_num.max())
    plt.plot(dates.num2date(x_fit), fit(x_fit), "r--")

    ax = plt.gca()
    plt.grid(visible=True, which='major', color='#666666', linestyle='-')
    plt.grid(visible=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
    years = plt.matplotlib.dates.YearLocator()
    months = plt.matplotlib.dates.MonthLocator()
    # yearsFmt = plt.matplotlib.dates.DateFormatter('%Y')
    ax.xaxis.set_major_locator(years)
    ax.xaxis.set_minor_locator(months)
    # ax.xaxis.set_major_formatter(yearsFmt)

    plt.title(f"{ydata} across {len(start_date_array)} {name} sessions")
    plt.colorbar(label=cdata)
    plt.ylabel(ydata)
    if ylimdata is not None:
        plt.ylim(ylimdata)
    if yticksdata is not None:
        plt.yticks(yticksdata)
    plt.show()


In [ ]:
__plot_activity("swim", "100m Time", "Distance")
__plot_activity("swim", "Distance", "100m Time", ylimdata=[0, 1])
__plot_activity("run", "1k Time", "Exercise Load", yticksdata=[300, 360, 420, 480], ylimdata=[300, 480])
__plot_activity("run", "Distance", "1k Time", ylimdata=[2, 10.3])
__plot_activity("strength", "Exercise Load", "Moving Time")
